# Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import scipy.sparse as sp

from sklearn.metrics import roc_auc_score, roc_curve, auc, f1_score, jaccard_score

import networkx as nx
# import dgl

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.datasets import Planetoid
from torch_geometric.transforms import NormalizeFeatures
import torch_geometric.transforms as T
from torch_geometric.nn import GCNConv
from torch_geometric.utils import negative_sampling
from torch_geometric.utils.convert import from_networkx

In [2]:
# Setting up GPU
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = torch.device('cpu')

# Graph Creation

In [61]:
node = pd.read_csv('node_information.csv', header=None)
train = pd.read_csv('train.txt', header=None, names=['Source','Target','Edge'], delim_whitespace=True)
test = pd.read_csv('test.txt', header=None, names=['Source','Target'], delim_whitespace=True)

In [4]:
node.loc[:, node.columns != node.columns[0]] = node.loc[:, node.columns != node.columns[0]].astype(int)
node.head()

,0,1,2,3,4,5,6,7,8,9,...,923,924,925,926,927,928,929,930,931,932
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,6,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
nodes = [i for i in node[0]]
nodes[:10]

[0, 4, 5, 6, 7, 9, 10, 11, 13, 17]

In [6]:
train.head()

,Source,Target,Edge
0,939,3809,1
1,2442,5784,1
2,179,3809,1
3,857,2280,1
4,1358,5722,1


In [8]:
# Creates the oriented graph
diG=nx.DiGraph()

# add the list of all nodes
diG.add_nodes_from(nodes)

# adds the corresponding links between two nodes in trainset
for i in range(train.shape[0]):
    if train['Edge'][i] == 1:
        diG.add_edge(train['Source'][i], train['Target'][i])

# Create a dictionary containing node features
node_attributes = {}
for _, row in node.iterrows():
    node_attributes[row[0]] = torch.from_numpy(np.array(row[1:])).to(torch.float)  

# Add the attributes to nodes
nx.set_node_attributes(diG, node_attributes, "x")

# Checks the number of edges and creates the non-oriented graph G
G = nx.Graph(diG)

print("The number of nodes: {}".format(G.number_of_nodes()))
print("The number of edges: {}".format(G.number_of_edges()))

The number of nodes: 3597
The number of edges: 5248


In [9]:
# Transform networkx graph to PyG graph
G = from_networkx(G)
print(G)

Data(x=[3597, 932], edge_index=[2, 10465])


For some reason there is an annoying duplication in edge and the edge index doesn't really follow what was originally put in the graph.

In [10]:
G.edge_index

tensor([[   0,    0,    1,  ..., 3595, 3595, 3596],
        [ 439, 1096, 2949,  ...,  537, 1518, 1323]])

In [11]:
def show_graph_stats(graph):
    
    print(f"Number of nodes: {graph.x.shape[0]}")
    print(f"Number of node features: {graph.x.shape[1]}")
    print(f"Number of edges: {graph.edge_index.shape[1]}")
    
show_graph_stats(G)

Number of nodes: 3597
Number of node features: 932
Number of edges: 10465


# Train-Val Split

In [51]:
split = T.RandomLinkSplit(
    num_val=0.2,
    num_test=0,
    is_undirected=True,
    add_negative_train_samples=True,
    neg_sampling_ratio=1.0,
)
train_data, val_data, _ = split(G)
print('train_data:', train_data)
print('val_data:', val_data)

train_data: Data(x=[3597, 932], edge_index=[2, 8398], edge_label=[8398], edge_label_index=[2, 8398])
val_data: Data(x=[3597, 932], edge_index=[2, 8398], edge_label=[2098], edge_label_index=[2, 2098])


# GNN

In [52]:
class Net(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super().__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, out_channels)

    def encode(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        return self.conv2(x, edge_index)

    def decode(self, z, edge_label_index):
        return (z[edge_label_index[0]] * z[edge_label_index[1]]).sum(
            dim=-1
        )  # product of a pair of nodes on each edge

    def decode_all(self, z):
        prob_adj = z @ z.t()
        return (prob_adj > 0).nonzero(as_tuple=False).t()
    

def train_link_predictor(
    model, train_data, val_data, optimizer, criterion, n_epochs=200
):

    for epoch in range(1, n_epochs + 1):

        model.train()
        optimizer.zero_grad()
        z = model.encode(train_data.x, train_data.edge_index)

        # sampling training negatives for every training epoch
        neg_edge_index = negative_sampling(
            edge_index=train_data.edge_index, num_nodes=train_data.num_nodes,
            num_neg_samples=train_data.edge_label_index.size(1), method='sparse')

        edge_label_index = torch.cat(
            [train_data.edge_label_index, neg_edge_index],
            dim=-1,
        )
        edge_label = torch.cat([
            train_data.edge_label,
            train_data.edge_label.new_zeros(neg_edge_index.size(1))
        ], dim=0)

        out = model.decode(z, edge_label_index).view(-1)
        loss = criterion(out, edge_label)
        loss.backward()
        optimizer.step()

        val_auc = eval_link_predictor(model, val_data)

        if epoch % 10 == 0:
            print(f"Epoch: {epoch:03d}, Train Loss: {loss:.3f}, Val AUC: {val_auc:.3f}")

    return model


@torch.no_grad()
def eval_link_predictor(model, data):

    model.eval()
    z = model.encode(data.x, data.edge_index)
    out = model.decode(z, data.edge_label_index).view(-1).sigmoid()

    return roc_auc_score(data.edge_label.cpu().numpy(), out.cpu().numpy())

In [53]:
model = Net(G.x.shape[1], 128, 64).to(device)
optimizer = torch.optim.Adam(params=model.parameters(), lr=0.01)
criterion = torch.nn.BCEWithLogitsLoss()

model = train_link_predictor(model, train_data, val_data, optimizer, criterion)

Epoch: 010, Train Loss: 0.694, Val AUC: 0.604
Epoch: 020, Train Loss: 0.686, Val AUC: 0.572
Epoch: 030, Train Loss: 0.662, Val AUC: 0.520
Epoch: 040, Train Loss: 0.634, Val AUC: 0.547
Epoch: 050, Train Loss: 0.607, Val AUC: 0.546
Epoch: 060, Train Loss: 0.578, Val AUC: 0.542
Epoch: 070, Train Loss: 0.557, Val AUC: 0.532
Epoch: 080, Train Loss: 0.541, Val AUC: 0.538
Epoch: 090, Train Loss: 0.529, Val AUC: 0.534
Epoch: 100, Train Loss: 0.519, Val AUC: 0.531
Epoch: 110, Train Loss: 0.509, Val AUC: 0.529
Epoch: 120, Train Loss: 0.501, Val AUC: 0.538
Epoch: 130, Train Loss: 0.495, Val AUC: 0.532
Epoch: 140, Train Loss: 0.496, Val AUC: 0.531
Epoch: 150, Train Loss: 0.486, Val AUC: 0.533
Epoch: 160, Train Loss: 0.480, Val AUC: 0.525
Epoch: 170, Train Loss: 0.481, Val AUC: 0.524
Epoch: 180, Train Loss: 0.477, Val AUC: 0.533
Epoch: 190, Train Loss: 0.472, Val AUC: 0.543
Epoch: 200, Train Loss: 0.468, Val AUC: 0.546


# Make Predictions

In [54]:
# Creates the oriented graph
G_test=nx.DiGraph()

# add the list of all nodes
G_test.add_nodes_from(nodes)

# adds the corresponding links between two nodes in trainset
for i in range(test.shape[0]):
        G_test.add_edge(test['Source'][i], test['Target'][i])

# Checks the number of edges and creates the non-oriented graph G
G_test = nx.Graph(G_test)

# Create a dictionary containing node features
node_attributes = {}
for _, row in node.iterrows():
    node_attributes[row[0]] = torch.from_numpy(np.array(row[1:])).to(torch.float)  

nx.set_node_attributes(G_test, node_attributes, "x")

G_test = from_networkx(G_test)
print(G_test)

Data(x=[3597, 932], edge_index=[2, 6979])


In [55]:
split = T.RandomLinkSplit(
    num_val=0,
    num_test=0,
    is_undirected=True,
    add_negative_train_samples=True,
    neg_sampling_ratio=1.0,
)
test_data, _, _ = split(G_test)
test_data

Data(x=[3597, 932], edge_index=[2, 6996], edge_label=[6996], edge_label_index=[2, 6996])

In [56]:
@torch.no_grad()
def test_link_predictor(model, data):

    model.eval()
    z = model.encode(data.x, data.edge_index)
    out = model.decode(z, data.edge_label_index).view(-1).sigmoid()

    return out, roc_auc_score(data.edge_label.cpu().numpy(), out.cpu().numpy())

In [57]:
test_preds, test_auc = test_link_predictor(model, test_data)
print(f"Test: {test_auc:.3f}")

Test: 0.913


In [21]:
test_predictions = pd.DataFrame(columns = ['Source', 'Target', 'Edge'], dtype=object)
# Iterate through rows
for i in range(test_preds.shape[0]-1):
    source_val = test_data['edge_index'][0][i].item() 
    target_val = test_data['edge_index'][1][i].item()
    edge_val = 1 if test_preds[i].item() >= 0.5 else 0
    
    # Add the row to the DataFrame
    test_predictions = test_predictions.append({
        'Source': source_val,
        'Target': target_val,
        'Edge': edge_val
    }, ignore_index=True)

C:\Users\Davina Chen\AppData\Local\Temp\ipykernel_28336\3638932465.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_predictions = test_predictions.append({
C:\Users\Davina Chen\AppData\Local\Temp\ipykernel_28336\3638932465.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_predictions = test_predictions.append({
C:\Users\Davina Chen\AppData\Local\Temp\ipykernel_28336\3638932465.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_predictions = test_predictions.append({
C:\Users\Davina Chen\AppData\Local\Temp\ipykernel_28336\3638932465.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_predictions = test_predictio

In [59]:
test_predictions.head()

,Source,Target,Edge
0,2067,2399,1
1,734,2664,1
2,1240,3432,1
3,20,1991,0
4,1619,3428,1


In [62]:
test.head()

,Source,Target
0,3425,4524
1,1620,2617
2,4832,6317
3,4984,7298
4,385,5481


In [63]:
# Join the two DataFrames
preds = pd.merge(test, test_predictions, on=['Source', 'Target'])

# Add the 'Edge' column to the original DataFrame
test['Edge'] = preds['Edge']

In [64]:
preds

,Source,Target,Edge
0,2329,2713,1


# Submission

In [48]:
submission = pd.DataFrame()
submission['ID'] = list(range(preds.shape[0]))
submission['Predicted'] = list(preds['Edge'])
submission.to_csv('submission.csv', index = False)